In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
from multiprocessing import Process,Queue,Event,Pipe

from Functions.check import check_sudoku,timer

In [2]:
# the function is not working weel (some bug)

def debug(msg='defaul'):
    print "{0}".format(msg)

def create_lines(q,event,read_msg):
    while True:
        np.random.seed() # start each process with diferent seed for random
        line_rdm = np.random.randint(1,10,size=(1,3)) # get one line of "sudoku"
        if len(np.unique(line_rdm)) == 3: # don't repeat numbers
            q.put(line_rdm) # add to Queue
            debug('processo pra criar')
            event.wait() # wait for process "control_process" finished
            debug('passei do wait')
            print read_msg.poll()
            debug('passei da mensagem')
            if read_msg.poll() is 'finished': # if the matrix is like Sudoku, then receive from pipe the message 'finished' 
                break #finish the process
            

def control_process(q,event,send_msg):
    time.sleep(0.1) # time for process's create_lines start first
    while True:
        while not q.full():pass # wait 3 process put the line of matrix in Queue
        list_tmp = []
        while not q.empty():
            list_tmp.append(q.get()) # 'create' de matrix 
        
        print list_tmp
        if len(np.unique(list_tmp)) == 9: # check if the matrix is Sudoku (don't repeat number)
            debug('consegui')
            q.put(list_tmp) # put the matrix in Queue for be caught in the future
            send_msg.send_bytes('finished') # send for 3 processes the message that finished 
            send_msg.close()
            event.set() # wake up 3 processes
            break #finish the process
        else:
            debug('não consegui') # don't find the matrix is Sudoku because some number was repeated
            event.set()# wake up 3 processes

In [3]:
q = Queue(3)
event = Event()
send,read = Pipe()


list_process = [Process(target=create_lines,args=(q,event,read)) for i in range(3)]
list_process.append(Process(target=control_process,args=(q,event,send)))
for process in list_process:
    process.start()

for process in list_process:
    process.join()

print q.get() #print the Sudoku